In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/home/cucchi/phd/devel/phd/')

In [ ]:
import os
import xarray as xr
import math
import numpy as np
import matplotlib.pyplot as plt

from lab import analysis
from lab.simulation import forcings

In [ ]:
REF_RESP_PATH = [
    '/home/cucchi/phd/data/response/lorenz96/rk4/SF_8_1.0_0/',
    '/home/cucchi/phd/data/response/lorenz96/rk4/SF_8_-1.0_0/'            
]

## Multiple Plots 

In [ ]:
force_intensity_delta = 3.0
forcing = 'SF_8_{}_0'.format(force_intensity_delta)

### Bins

In [ ]:
obs_all = ['energy_bin_{}q_{}q'.format(str(np.round(quant, 1)), str(np.round(quant + 0.1, 1))) for quant in np.arange(0, 0.9, 0.1)]
obs_all.append('energy_exceed_0.9q')

Compute **susceptibilities**

In [ ]:
chi_semidiff = {}
chi_normal = {}
for obs in obs_all:
    response_p1 = xr.open_dataarray(os.path.join(REF_RESP_PATH[0], 'response_lorenz96_rk4_{}_SF_8_1.0_0.nc'.format(obs)))
    response_m1 = xr.open_dataarray(os.path.join(REF_RESP_PATH[1], 'response_lorenz96_rk4_{}_SF_8_-1.0_0.nc'.format(obs)))
    response_semidiff = (response_p1 - response_m1)/2
    chi_semidiff[obs] = analysis.compute_susceptibility(response_semidiff.values.squeeze())
    chi_normal[obs] = analysis.compute_susceptibility(response_p1.values.squeeze())

Compute **predicted responses**

In [ ]:
resp_real = {}
resp_pred_semidiff = {}
resp_pred_normal = {}
for obs in obs_all:
    resp_real[obs] = xr.open_dataarray(
        '/home/cucchi/phd/data/response/lorenz96/rk4/{f}/response_lorenz96_rk4_{o}_{f}.nc'.format(
            o=obs,
            f=forcing
        )
    )
    resp_pred_semidiff[obs] = analysis.compute_response(chi_semidiff[obs], forcings.StepForcing(
        force_intensity_delta=force_intensity_delta
    ))
    resp_pred_normal[obs] = analysis.compute_response(chi_normal[obs], forcings.StepForcing(
        force_intensity_delta=force_intensity_delta
    ))

In [ ]:
fig, ax = plt.subplots(5, 2, sharex='all', sharey='all', figsize=(15, 20))

titles = [
    'freq. of energy val. below 10th perc.',
    'freq. of energy val. between 10th and 20th perc.',
    'freq. of energy val. between 20th and 30th perc.',
    'freq. of energy val. between 30th and 40th perc.',
    'freq. of energy val. between 40th and 50th perc.',
    'freq. of energy val. between 50th and 60th perc.',
    'freq. of energy val. between 60th and 70th perc.',
    'freq. of energy val. between 70th and 80th perc.',
    'freq. of energy val. between 80th and 90th perc.',
    'freq. of energy val. above 90th percentile',
]

k = 0
for i in range(5):
    for j in range(2):
        l1, = ax[i, j].plot(np.arange(0, 100, 0.01), resp_real[obs_all[k]].values, alpha=0.5)
        l3, = ax[i, j].plot(np.arange(0, 100, 0.01), resp_pred_normal[obs_all[k]][0:10000], alpha=0.8, color='m')
        l2, = ax[i, j].plot(np.arange(0, 100, 0.01), resp_pred_semidiff[obs_all[k]][0:10000], alpha=0.8)
        ax[i, j].set_title(titles[k], size=17)
        ax[i, j].set_ylim([-0.15, 0.15])
        ax[i, j].tick_params(labelsize='x-large')
        std_real = resp_real[obs_all[k]].std().values
        avg_real = resp_real[obs_all[k]].mean().values
        sig_digit_real = -math.floor(math.log10(std_real))
        std_pred_normal = resp_pred_normal[obs_all[k]][0:10000].std()
        avg_pred_normal = resp_pred_normal[obs_all[k]][0:10000].mean()
        sig_digit_pred_normal = -math.floor(math.log10(std_pred_normal))
        std_pred_semidiff = resp_pred_semidiff[obs_all[k]][0:10000].std()
        avg_pred_semidiff = resp_pred_semidiff[obs_all[k]][0:10000].mean()
        sig_digit_pred_semidiff = -math.floor(math.log10(std_pred_semidiff))
        
        ax[i, j].text(48, -0.13, 
                      'Averages:\nreal:              {} +/- {}\npred semid:  {} +/- {}\npred normal: {} +/- {}'.format(
                          round(float(avg_real), sig_digit_real),
                          round(float(std_real), sig_digit_real),
                          round(float(avg_pred_semidiff), sig_digit_pred_semidiff),
                          round(float(std_pred_semidiff), sig_digit_pred_semidiff),
                          round(float(avg_pred_normal), sig_digit_pred_normal),
                          round(float(std_pred_normal), sig_digit_pred_normal),
                      ), 
                      size = 15,
                      bbox={'facecolor':'white', 'pad':5, 'alpha':0.5}
                     )
        
        if k==1:
            ax[i, j].legend((l1, l2, l3), ('real', 'predicted (semidiff)', 'predicted (normal)'), loc='upper right', shadow=True, fontsize='x-large')
        if not j%2:
            ax[i, j].set_ylabel('frequency', size=15)
        if i==4:
            ax[i, j].set_xlabel('time', size=15)
            
        k += 1
        
plt.tight_layout()
plt.savefig('/home/cucchi/phd/devel/phd/logbook/fig/pred_bin_S_{}.png'.format(
    str(force_intensity_delta).replace('.','')
))

### Below

In [ ]:
obs_all = ['energy_bin_0.0q_{}q'.format(str(np.round(quant + 0.1, 1))) for quant in np.arange(0, 0.9, 0.1)]
obs_all.append('energy_exceed_0.9q')

Compute **susceptibilities**

In [ ]:
chi_semidiff = {}
chi_normal = {}
for obs in obs_all:
    response_p1 = xr.open_dataarray(os.path.join(REF_RESP_PATH[0], 'response_lorenz96_rk4_{}_SF_8_1.0_0.nc'.format(obs)))
    response_m1 = xr.open_dataarray(os.path.join(REF_RESP_PATH[1], 'response_lorenz96_rk4_{}_SF_8_-1.0_0.nc'.format(obs)))
    response_semidiff = (response_p1 - response_m1)/2
    chi_semidiff[obs] = analysis.compute_susceptibility(response_semidiff.values.squeeze())
    chi_normal[obs] = analysis.compute_susceptibility(response_p1.values.squeeze())

Compute **predicted responses**

In [ ]:
resp_real = {}
resp_pred_semidiff = {}
resp_pred_normal = {}
for obs in obs_all:
    resp_real[obs] = xr.open_dataarray(
        '/home/cucchi/phd/data/response/lorenz96/rk4/{f}/response_lorenz96_rk4_{o}_{f}.nc'.format(
            o=obs,
            f=forcing
        )
    )
    resp_pred_semidiff[obs] = analysis.compute_response(chi_semidiff[obs], forcings.StepForcing(
        force_intensity_delta=force_intensity_delta
    ))
    resp_pred_normal[obs] = analysis.compute_response(chi_normal[obs], forcings.StepForcing(
        force_intensity_delta=force_intensity_delta
    ))

In [ ]:
fig, ax = plt.subplots(5, 2, sharex='all', sharey='all', figsize=(15, 20))

titles = [
    'freq. of energy val. below 10th perc.',
    'freq. of energy val. below 20th perc.',
    'freq. of energy val. below 30th perc.',
    'freq. of energy val. below 40th perc.',
    'freq. of energy val. below 50th perc.',
    'freq. of energy val. below 60th perc.',
    'freq. of energy val. below 70th perc.',
    'freq. of energy val. below 80th perc.',
    'freq. of energy val. below 90th perc.',
    'freq. of energy val. above 90th perc.',
]
k = 0
for i in range(5):
    for j in range(2):
        l1, = ax[i, j].plot(np.arange(0, 100, 0.01), resp_real[obs_all[k]].values, alpha=0.5)
        l3, = ax[i, j].plot(np.arange(0, 100, 0.01), resp_pred_normal[obs_all[k]][0:10000], alpha=0.8, color='m')
        l2, = ax[i, j].plot(np.arange(0, 100, 0.01), resp_pred_semidiff[obs_all[k]][0:10000], alpha=0.8)
        ax[i, j].set_title(titles[k], size=17)
        ax[i, j].set_ylim([-0.18, 0.18])
        ax[i, j].tick_params(labelsize='x-large')
        std_real = resp_real[obs_all[k]].std().values
        avg_real = resp_real[obs_all[k]].mean().values
        sig_digit_real = -math.floor(math.log10(std_real))
        std_pred_normal = resp_pred_normal[obs_all[k]][0:10000].std()
        avg_pred_normal = resp_pred_normal[obs_all[k]][0:10000].mean()
        sig_digit_pred_normal = -math.floor(math.log10(std_pred_normal))
        std_pred_semidiff = resp_pred_semidiff[obs_all[k]][0:10000].std()
        avg_pred_semidiff = resp_pred_semidiff[obs_all[k]][0:10000].mean()
        sig_digit_pred_semidiff = -math.floor(math.log10(std_pred_semidiff))
        """
        ax[i, j].text(48, -0.13, 
                      'Averages:\nreal:              {} +/- {}\npred semid:  {} +/- {}\npred normal: {} +/- {}'.format(
                          round(float(avg_real), sig_digit_real),
                          round(float(std_real), sig_digit_real),
                          round(float(avg_pred_semidiff), sig_digit_pred_semidiff),
                          round(float(std_pred_semidiff), sig_digit_pred_semidiff),
                          round(float(avg_pred_normal), sig_digit_pred_normal),
                          round(float(std_pred_normal), sig_digit_pred_normal),
                      ), 
                      size = 15,
                      bbox={'facecolor':'white', 'pad':5, 'alpha':0.5}
                     )
        """
        if k==1:
            ax[i, j].legend((l1, l2, l3), ('real', 'predicted (semidiff)', 'predicted (normal)'), loc='upper right', shadow=True, fontsize='x-large')
        if not j%2:
            ax[i, j].set_ylabel('frequency', size=15)
        if i==4:
            ax[i, j].set_xlabel('time', size=15)
            
        k += 1
        
plt.tight_layout()
plt.savefig('/home/cucchi/phd/devel/phd/logbook/fig/pred_below_S_{}.png'.format(
    str(force_intensity_delta).replace('.','')
))

## Single Plot

In [ ]:
force_intensity_delta = 3.0
forcing = 'SF_8_{}_0'.format(force_intensity_delta)
obs = 'energy_exceed_0.99q'

In [ ]:
response_p1 = xr.open_dataarray(os.path.join(REF_RESP_PATH[0], 'response_lorenz96_rk4_{}_SF_8_1.0_0.nc'.format(obs)))
response_m1 = xr.open_dataarray(os.path.join(REF_RESP_PATH[1], 'response_lorenz96_rk4_{}_SF_8_-1.0_0.nc'.format(obs)))
response_semidiff = (response_p1 - response_m1)/2
chi_semidiff = analysis.compute_susceptibility(response_semidiff.values.squeeze())
chi_normal = analysis.compute_susceptibility(response_p1.values.squeeze())

resp_real = xr.open_dataarray(
    '/home/cucchi/phd/data/response/lorenz96/rk4/{f}/response_lorenz96_rk4_{o}_{f}.nc'.format(
        o=obs,
        f=forcing
    )
)
resp_pred_semidiff = analysis.compute_response(chi_semidiff, forcings.StepForcing(
    force_intensity_delta=force_intensity_delta,
))
resp_pred_normal = analysis.compute_response(chi_normal , forcings.StepForcing(
    force_intensity_delta=force_intensity_delta,
))

### Plot

In [ ]:
l1, = plt.plot(np.arange(0, 100, 0.01), resp_real.values, alpha=0.5)
l2, = plt.plot(np.arange(0, 100, 0.01), resp_pred_semidiff[0:10000], alpha=0.7)
l3, = plt.plot(np.arange(0, 100, 0.01), resp_pred_normal[0:10000], color='m', alpha=0.7)
#plt.set_title(titles[k], size=17)
plt.ylim(-0.10, 0.10)
#plt.tick_params(labelsize='x-normal')
std_real = resp_real.std().values
avg_real = resp_real.mean().values
sig_digit_real = -math.floor(math.log10(std_real))
std_pred_normal = resp_pred_normal[0:10000].std()
avg_pred_normal = resp_pred_normal[0:10000].mean()
sig_digit_pred_normal = -math.floor(math.log10(std_pred_normal))
std_pred_semidiff = resp_pred_semidiff[0:10000].std()
avg_pred_semidiff = resp_pred_semidiff[0:10000].mean()
sig_digit_pred_semidiff = -math.floor(math.log10(std_pred_semidiff))
plt.legend((l1, l2, l3), ('real', 'predicted (semidiff)', 'predicted (normal)'), loc='lower right', shadow=True)
plt.title('frequency of energy val. above 99th perc.')
plt.ylabel('frequency')
plt.xlabel('time')
plt.savefig('/home/cucchi/phd/devel/phd/logbook/fig/pred_{}_S_{}.png'.format(
    obs.replace('.', ''), str(force_intensity_delta).replace('.', '')
))